In [2]:
import investpy
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
import numpy as np
from statistics import mean

In [3]:
def extract_trend_logic(df, window_size=5, trend_limit=3, identify='both'):
    """
    This function identifies up and down trends in a given DataFrame containing historical stock prices.
    
    Args:
        df (:obj:pandas.DataFrame): DataFrame containing 'Close' prices indexed by date.
        window_size (:obj:int, optional): Number of consecutive days to consider for a trend.
        trend_limit (:obj:int, optional): Maximum number of trends to identify.
        identify (:obj:str, optional): Type of trends to identify ('both', 'up', 'down').

    Returns:
        :obj:pandas.DataFrame: DataFrame with trend labels for 'Up Trend' and 'Down Trend'.
    """

    objs = []
    up_trend = {
        'name': 'Up Trend',
        'element': np.negative(df['Close'])
    }
    down_trend = {
        'name': 'Down Trend',
        'element': df['Close']
    }

    if identify == 'both':
        objs.append(up_trend)
        objs.append(down_trend)
    elif identify == 'up':
        objs.append(up_trend)
    elif identify == 'down':
        objs.append(down_trend)

    results = {}

    for obj in objs:
        trends = []
        values = []
        limit = None
        from_trend = 0

        for index, value in enumerate(obj['element']):
            if limit is not None:
                if limit > value:  # Downward movement
                    values.append(value)
                    limit = mean(values)
                elif limit < value:  # Upward movement or reset
                    if len(values) >= window_size:
                        min_value = min(values)
                        to_trend = from_trend + values.index(min_value)
                        trends.append({
                            'from': df.index[from_trend],
                            'to': df.index[to_trend]
                        })
                    limit = None
                    values = []
            else:
                from_trend = index
                values.append(value)
                limit = mean(values)

        results[obj['name']] = trends

    # Handling overlapping trends and limiting results
    if identify == 'both':
        up_trends = results['Up Trend']
        down_trends = results['Down Trend']
        up_labels = [chr(65 + i) for i in range(min(trend_limit, len(up_trends)))]
        down_labels = [chr(65 + i) for i in range(min(trend_limit, len(down_trends)))]

        for up_trend, up_label in zip(up_trends, up_labels):
            df.loc[up_trend['from']:up_trend['to'], 'Up Trend'] = up_label

        for down_trend, down_label in zip(down_trends, down_labels):
            df.loc[down_trend['from']:down_trend['to'], 'Down Trend'] = down_label

    elif identify == 'up':
        up_trends = results['Up Trend']
        up_labels = [chr(65 + i) for i in range(min(trend_limit, len(up_trends)))]
        
        for up_trend, up_label in zip(up_trends, up_labels):
            df.loc[up_trend['from']:up_trend['to'], 'Up Trend'] = up_label

    elif identify == 'down':
        down_trends = results['Down Trend']
        down_labels = [chr(65 + i) for i in range(min(trend_limit, len(down_trends)))]

        for down_trend, down_label in zip(down_trends, down_labels):
            df.loc[down_trend['from']:down_trend['to'], 'Down Trend'] = down_label

    return df       



In [4]:
def get_stock_data(ticker,start_date,end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    return stock_data


In [5]:
stock_df=extract_trend_logic(get_stock_data('TSLA',datetime.now()-timedelta(days=100),datetime.now()))
stock_df.fillna(0,inplace=True)

[*********************100%***********************]  1 of 1 completed


In [12]:
stock_df[stock_df['Up Trend']=="A"]

,Open,High,Low,Close,Adj Close,Volume,Up Trend,Down Trend
Date,,,,,,,,
2024-08-15,205.020004,215.880005,204.820007,214.139999,214.139999,89848500,A,0
2024-08-16,211.149994,219.800003,210.800003,216.119995,216.119995,88765100,A,0
2024-08-19,217.070007,222.979996,214.089996,222.720001,222.720001,76435200,A,0
2024-08-20,224.880005,228.220001,219.559998,221.100006,221.100006,74001200,A,0
2024-08-21,222.669998,224.660004,218.860001,223.270004,223.270004,70146000,A,0


In [16]:
# isinstance(stock_df['Up Trend'].tail(1),object)
# last_element=stock_df['Up Trend'].iloc[-1]
# if isinstance(last_element, int):
#     print("The last element is an integer.")
# elif isinstance(last_element, str):
#     print("The last element is a string.")
# else:
#     print("The last element is neither an integer nor a string.")

last_df=[]
last_element=stock_df['Up Trend'].iloc[-1]
if isinstance(last_element, int):
    last_df.append(stock_df.iloc[-1])
else:
    print('last_element is different')

In [17]:
last_df

[Open          258.040009
 High          263.350006
 Low           255.820007
 Close         257.549988
 Adj Close     257.549988
 Volume          53757000
 Up Trend               0
 Down Trend             0
 Name: 2024-10-30 00:00:00, dtype: object]

In [1]:
asset_map = {
        'jpy': 'JPY%3DX',
        'eur': 'EURUSD%3DX',
        'gbp': 'GBPUSD%3DX',
        'franc': 'CHFUSD%3DX',
        'us100': 'NQ%3DF',
        'us30': 'YM%3DF',
        'us500': 'ES%3DF',
        'crude oil': 'CL%3DF',
        'gold': 'GC%3DF',
        'bitcoin': 'BTC-USD',
        'Apple': 'AAPL',
        'Microsoft': 'MSFT',
        'Google': 'GOOGL',
        'Amazon': 'AMZN',
        'NVIDIA': 'NVDA',
        'Berkshire': 'BRK-B',
        'Meta': 'META',
        'Tesla': 'TSLA',
        'UGI': 'UNH',
        'Exxon': 'XOM',
        'Johnson&Johnson': 'JNJ',
        'Visa': 'V',
        'Procter&Gamble': 'PG',
        'JPMorgan': 'JPM',
        'Mastercard': 'MA',
        'EliLilly': 'LLY',
        'Chevron': 'CVX',
        'HomeDepot': 'HD',
        'Pfizer': 'PFE',
        'AbbVie': 'ABBV',
        'Merck': 'MRK',
        'Pepsi.': 'PEP',
        'Coca-Cola': 'KO',
        'Broadcom': 'AVGO',
        'Costco': 'COST',
        'WaltDisney': 'DIS',
        'Comcast': 'CMCSA',
        'Cisco': 'CSCO',
        'Intel': 'INTC',
        'Salesforce': 'CRM',
        'TexasInstruments ': 'TXN',
        'ThermoFisher ': 'TMO',
        'Bristol-Myers Squibb': 'BMY',
        'Verizon': 'VZ',
        'Nike': 'NKE',
        'Oracle': 'ORCL',
        'NextEra Energy': 'NEE',
        'McDonald ': 'MCD',
        'Adobe': 'ADBE',
        'PhilipMorris ': 'PM',
        'AT&T': 'T',
        'AbbottLaboratories': 'ABT',
        'AdvancedMicro': 'AMD',
        'UnionPacific': 'UNP',
        'Medtronic': 'MDT',
        'CVSHealth': 'CVS',
        'Qualcomm': 'QCOM',
        'Honeywell': 'HON',
        'GoldmanSachs ': 'GS'
    }

In [54]:
for i in asset_map:
    print(i)

jpy
eur
gbp
franc
us100
us30
us500
crude oil
gold
bitcoin
Apple
Microsoft
Google
Amazon
NVIDIA
Berkshire
Meta
Tesla
UGI
Exxon
Johnson&Johnson
Visa
Procter&Gamble
JPMorgan
Mastercard
EliLilly
Chevron
HomeDepot
Pfizer
AbbVie
Merck
Pepsi.
Coca-Cola
Broadcom
Costco
WaltDisney
Comcast
Cisco
Intel
Salesforce
TexasInstruments 
ThermoFisher 
Bristol-Myers Squibb
Verizon
Nike
Oracle
NextEra Energy
McDonald 
Adobe
PhilipMorris 
AT&T
AbbottLaboratories
AdvancedMicro
UnionPacific
Medtronic
CVSHealth
Qualcomm
Honeywell
GoldmanSachs 


In [70]:
asset_map.keys()

dict_keys(['jpy', 'eur', 'gbp', 'franc', 'us100', 'us30', 'us500', 'crude oil', 'gold', 'bitcoin', 'Apple', 'Microsoft', 'Google', 'Amazon', 'NVIDIA', 'Berkshire', 'Meta', 'Tesla', 'UGI', 'Exxon', 'Johnson&Johnson', 'Visa', 'Procter&Gamble', 'JPMorgan', 'Mastercard', 'EliLilly', 'Chevron', 'HomeDepot', 'Pfizer', 'AbbVie', 'Merck', 'Pepsi.', 'Coca-Cola', 'Broadcom', 'Costco', 'WaltDisney', 'Comcast', 'Cisco', 'Intel', 'Salesforce', 'TexasInstruments ', 'ThermoFisher ', 'Bristol-Myers Squibb', 'Verizon', 'Nike', 'Oracle', 'NextEra Energy', 'McDonald ', 'Adobe', 'PhilipMorris ', 'AT&T', 'AbbottLaboratories', 'AdvancedMicro', 'UnionPacific', 'Medtronic', 'CVSHealth', 'Qualcomm', 'Honeywell', 'GoldmanSachs '])

KeyError: 2